<img src="../../../img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">Introduction to the SpatioTemporal Asset Catalog (STAC) in R</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/R/stac/stac-introduction.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Felipe Carvalho de Souza<sup><a href="https://orcid.org/0000-0002-5826-1700"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: March 24, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook gives an overview on how to use the STAC service to discover and access the data products from the <em>Brazil Data Cube</em>. This version uses the <b>R client package</b> named <a href="https://github.com/brazil-data-cube/rstac" target="_blank">rstac</a>.
</div>    

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Zaglia, M.; Vinhas, L.; Queiroz, G. R.; Simões, R. <a href="http://urlib.net/rep/8JMKD3MGPDW34R/3UFEFD8" target="_blank">Catalogação de Metadados do Cubo de Dados do Brasil com o SpatioTemporal Asset Catalog</a>. In: Proceedings XX GEOINFO, November 11-13, 2019, São José dos Campos, SP, Brazil. p 280-285.
    </div>
</div>

# Introduction
<hr style="border:1px solid #0077b9;">

The [**S**patio**T**emporal **A**sset **C**atalog (STAC)](https://stacspec.org/) is a specification created through the colaboration of several organizations intended to increase satellite image search interoperability.

The diagram depicted in the picture contains the most important concepts behind the STAC data model:

<center>
<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac-model.png" width="480" />
<br/>
<b>Figura 1</b> - STAC model.
</center>    

# STAC API
<hr style="border:1px solid #0077b9;">

The STAC API is the dynamic version of a STAC. It returns `STAC Catalog`, `Collection`, `Item`, `ItemCollection` objects through several *endpoints*.

The description of the concepts below are adapted from the [STAC Specification](https://github.com/radiantearth/stac-spec):

- **Item**: a `STAC Item` is the atomic unit of metadata in STAC, providing links to the actual `assets` (including thumbnails) that they represent. It is a `GeoJSON Feature` with additional fields for things like time, links to related entities and mainly to the assets. According to the specification, this is the atomic unit that describes the data to be discovered in a `STAC Catalog` or `Collection`.

- **Asset**: a `spatiotemporal asset` is any file that represents information about the earth captured in a certain space and time.


- **Catalog**: provides a structure to link various `STAC Items` together or even to other `STAC Catalogs` or `Collections`.


- **Collection:** is a specialization of the `Catalog` that allows additional information about a spatio-temporal collection of data.

For running the examples in this Jupyter Notebook you will need to install the [STAC client for R](https://github.com/brazil-data-cube/rstac). To install it from the Brazil Data Cube's GitHub repository, you can use `devtools` with the following command:

In [ ]:
# require package - install from CRAN
#install.packages("rstac")

If you are not using the kaggle environment, please install the packages listed below:

In [ ]:
# Remove '#' to install the packages
# install.packages(c("magrittr", "tibble", "dplyr", "raster"), dependencies = FALSE)

Let's import the `rstac` package and complement package, as follows:

In [ ]:
library(rstac)    # Package rstac
library(magrittr) # Package to use pipe operator %>%
library(tibble)   # Package to represent data frame structure
library(dplyr)    # Package to handle with data frame
library(raster)   # Package to manipulate raster images

Then, let's create a `RSTACQuery` object called `stac` attached to the service address, which will allow us to comunicate to the given `STAC` service.

In [ ]:
stac_obj <- stac("https://brazildatacube.dpi.inpe.br/stac/")

# Listing the Available Data Products
<hr style="border:1px solid #0077b9;">

To list the available image collections and data cube collections we will make a request using the `get_request()` function.

In [ ]:
catalog <- stac_obj %>% get_request()

print(catalog)

In [ ]:
# visualizing as data frame
lapply(catalog$link, function(x){ list("title" = x$title, "link" = x$href) }) %>% 
    dplyr::bind_rows()

# Retrieving the Metadata of a Data Product
<hr style="border:1px solid #0077b9;">

The `collection` method returns information about a given image or data cube collection identified by its name. In this example we are retrieving inormation about the datacube collection `CB4_64_16D_STK-1`:

In [ ]:
collection <- stac_obj %>% collections("CB4_64_16D_STK-1") 
collection_info <- collection %>%  get_request()

print(collection_info)

# Retrieving data
<hr style="border:1px solid #0077b9;">

The `items()` function returns a consult given a `bbox` and `datetime`:

In [ ]:
item <- collection %>% items(datetime = "2018-08-01/2019-07-31",
                             bbox     = c(-46.62597656250001,-13.19716452328198,-45.03570556640626,-12.297068292853805),
                             limit    = 20) %>% 
                        get_request() 
                                        
                                  
print(item)

Let's create a dataframe with the links corresponding to each band, with the function `assets_list`.

In [ ]:
items_list <- item %>% assets_list(assets_names = c("BAND13", "BAND14", "BAND15")) %>% 
              tibble::as_tibble() %>% dplyr::mutate(path = paste0(path, "?access_token=change-me"))

In [ ]:
head(items_list)

Let's read the images corresponding to the first date.

In [ ]:
first_date_blue_url  <- items_list[items_list$band == "BAND13",][1,]
first_date_green_url <- items_list[items_list$band == "BAND14",][1,]
first_date_red_url   <- items_list[items_list$band == "BAND15",][1,]

In [ ]:
# reading the first images from each bands
blue_rast  <- raster::raster(first_date_blue_url$path)
green_rast <- raster::raster(first_date_green_url$path)
red_rast   <- raster::raster(first_date_red_url$path)

Let's crop the image with a 1 to 1500 in line and column.

In [ ]:
# croping using extent
blue_rast_cropped  <- (raster::crop(blue_rast, raster::extent(blue_rast, 1, 1500, 1, 1500)))
green_rast_cropped <- (raster::crop(green_rast, raster::extent(green_rast, 1, 1500, 1, 1500)))
red_rast_cropped   <- (raster::crop(red_rast, raster::extent(red_rast, 1, 1500, 1, 1500)))

Visualizing each band separately

In [ ]:
# setting plot display options
options(repr.plot.width = 16, repr.plot.height = 5)
par(mfrow = c(1, 3))

plot(blue_rast_cropped, main = "Blue Band")
plot(green_rast_cropped, main = "Green Band")
plot(red_rast_cropped, main = "Red Band")

# Visualizing Composed Images
<hr style="border:1px solid #0077b9;">

Let's create a band stack to create our composition plot.

In [ ]:
rgb <- raster::stack(red_rast_cropped, 
                     blue_rast_cropped, 
                     green_rast_cropped)

In [ ]:
plotRGB(rgb, r = 1, g = 2, b = 3, stretch = "lin")

# References
<hr style="border:1px solid #0077b9;">

- [Spatio Temporal Asset Catalog Specification](https://stacspec.org/)


- [Brazil Data Cube R Client Library for STAC Service - GitHub Repository](https://github.com/brazil-data-cube/rstac)